In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm

In [2]:
from scipy.stats import pearsonr,spearmanr

In [3]:
from gplearn.genetic import SymbolicRegressor,SymbolicTransformer
from gplearn import fitness
from gplearn.functions import make_function
from gplearn.fitness import make_fitness

In [4]:
post=ff.read('post')
filter0=ff.filter0
close=ff.read('close')*post*filter0
open_=ff.read('open')*post*filter0
high=ff.read('high')*post*filter0
low=ff.read('low')*post*filter0
vol=ff.read('vol')*filter0
amount=ff.read('amount')*post*filter0

In [5]:
close

,20100104,20100105,20100106,20100107,20100108,20100111,20100112,20100113,20100114,20100115,...,20240513,20240514,20240515,20240516,20240517,20240520,20240521,20240522,20240523,20240524
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1280.34161,1273.33883,1264.00179,1303.68421,1332.86246,1328.19394,1348.03515,1349.20228,1330.52820,1320.02403
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1348.24368,1355.51184,1404.57192,1486.33872,1635.33600,1668.04272,1669.85976,1708.01760,1737.09024,1633.51896
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.26560,46.49216,47.50816,45.92320,46.24832,48.52416,47.34560,47.14240,46.28896,44.74464
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,153.39640,155.78080,157.37040,161.34440,171.67680,166.11320,166.51060,168.89500,164.52360,160.15220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873706.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873726.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873806.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873833.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
mkv=(ff.read('mkv','factor')*filter0).loc[close.index,'20200207':'20210101']
mkv_s=mkv.values.reshape(-1)

In [7]:
inds = pd.read_pickle('/mydata2/wangs/data/dict_ind_matrix_sw1.pk')

In [8]:
mkt20=close.shift(-10,axis=1)/close - 1

In [9]:
stock_test=pd.DataFrame({'open':open_.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'close':close.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'high':high.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'low':low.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'vol':vol.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'amount':amount.loc[:,'20200101':'20210101'].values.reshape(-1),
                         'accelerated_turnover_rank_RC':pd.DataFrame(ff.read('accelerated_turnover_rank_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         # 'CSK_XYY_UP_DOWN_120D_RC':ff.read('CSK_XYY_UP_DOWN_120D_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'high_fre_vol_RC':ff.read('high_fre_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'high_fre_diff_vol_RC':ff.read('high_fre_diff_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'high_fre_absdiff_vol_RC':ff.read('high_fre_absdiff_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'peak_count_vol_RC':ff.read('peak_count_vol_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'overnightsmart20_RC':ff.read('overnightsmart20_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'CTR_RC': ff.read('CTR_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'jumpCTR_RC': ff.read('jumpCTR_RC').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'turnover_rate_proportion_l': ff.read('turnover_rate_proportion_l').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1),
                         # 'synergy':ff.read('synergy').loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1)
                        })
stock_code = list(np.repeat(list(ff.read('synergy').index),len(ff.read('synergy').loc[:,'20200207':'20210101'].columns)))
date_code = list(np.tile(list(ff.read('synergy').loc[:,'20200207':'20210101'].columns),len(list(ff.read('synergy').index))))
stock_test['code'] = stock_code
stock_test['date'] = date_code
stock_test['1'],stock_test['5'],stock_test['10'],stock_test['20'],stock_test['40'],stock_test['60']=1,5,10,20,40,60

In [10]:
test_mkt=mkt20.loc[ff.read('synergy').index,'20200207':'20210101'].values.reshape(-1)

In [11]:
ff.read('synergy').columns

Index(['20200102', '20200103', '20200106', '20200107', '20200108', '20200109',
       '20200110', '20200113', '20200114', '20200115',
       ...
       '20240409', '20240410', '20240411', '20240412', '20240415', '20240416',
       '20240417', '20240418', '20240419', '20240422'],
      dtype='object', length=1042)

In [12]:
stock_test

,open,close,high,low,vol,amount,code,date,1,5,10,20,40,60
0,1593.86740,1596.05078,1603.69261,1573.12529,924852.96,1.468381e+08,000001.SZ,20200207,1,5,10,20,40,60
1,1584.04219,1582.95050,1586.22557,1561.11670,1339495.24,2.109127e+08,000001.SZ,20200210,1,5,10,20,40,60
2,1593.86740,1614.60951,1630.98486,1589.50064,1407507.44,2.267652e+08,000001.SZ,20200211,1,5,10,20,40,60
3,1614.60951,1612.42613,1617.88458,1593.86740,1070503.21,1.717478e+08,000001.SZ,20200212,1,5,10,20,40,60
4,1605.87599,1599.32585,1624.43472,1594.95909,1013205.28,1.628068e+08,000001.SZ,20200213,1,5,10,20,40,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169853,NaN,NaN,NaN,NaN,NaN,NaN,688555.SH,20201225,1,5,10,20,40,60
1169854,NaN,NaN,NaN,NaN,NaN,NaN,688555.SH,20201228,1,5,10,20,40,60
1169855,NaN,NaN,NaN,NaN,NaN,NaN,688555.SH,20201229,1,5,10,20,40,60
1169856,NaN,NaN,NaN,NaN,NaN,NaN,688555.SH,20201230,1,5,10,20,40,60


In [13]:
unique_code=close.index.to_list()  # list
unique_date=close.loc[:,'20200101':'20210101'].columns.to_list()

In [18]:
fields=['open','close','high','low','vol','amount','1','5','20','40','60']

In [19]:
# fields=['open','close','high','low','vol','amount','accelerated_turnover_rank_RC','CSK_XYY_UP_DOWN_120D_RC','high_fre_vol_RC','high_fre_diff_vol_RC','high_fre_absdiff_vol_RC','peak_count_vol_RC','overnightsmart20_RC','CTR_RC','jumpCTR_RC','turnover_rate_proportion_l','synergy','1','5','20','40','60']

In [20]:
# 自带算子
init_function = ['add','sub','mul','div','sqrt','log','inv','abs','neg']

In [21]:
# 自定义算子
def _Add(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.add(data1,data2)
    
def _Sub(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.subtract(data1,data2)
    
def _Mul(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.multiply(data1,data2)
    
def _Div(data1,data2):
    def protect_division(data1,data2):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data2)>0.001,np.divide(data1,data2),0.)
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return protect_division(data1,data2)
    
def _Sqrt(data):
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return np.sqrt(np.abs(data))
    
def _Log(data):
    def protect_log(data):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data) > 0.001, np.log(np.abs(data)), 0.)
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return protect_log(data)
    
def _Inv(data):
    data=np.array(data)
    def protect_inv(data):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data) > 0.001, 1./data, 0.)
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return protect_inv(data)

def _ts_max(data,window): #历史rolling最大
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).max())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

def _ts_min(data,window): #历史rolling最小
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).min())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

def _ts_mid(data,window): #历史rolling中位数
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).median())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))


def _ts_mean(data,window):  #历史rolling平均
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))


def _ts_std(data,window): #历史rolling std
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).std())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _ts_sum(data,window): #历史rolling求和
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(value.values*window)
    except:
        return np.zeros(len(data))
    
    
def _ts_product(data,window): #历史rolling求积
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':np.log(data)})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(np.exp(value.values*window))
    except:
        return np.zeros(len(data))

    
def _delay(data,window):  # 几天以前的因子值
    window=window[0]
    if window not in [1,5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x : x.shift(window))
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

    
def _delta(data,window):  # 因子值与几天前的因子值之差
    window=window[0]
    if window not in [1,5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x : x.shift(window))
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

    
def _rank(data):  # 因子在截面上的分位数
    if len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('date')['data'].transform(lambda s : s.rank()/s.count())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _ts_rank(data,window):  # 因子在过去几天中的分位数
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code  
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).rank()/window)
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _sigmoid(data):
    data=np.array(data)
    with np.errstate(over='ignore',under='ignore'):
        return 1/(1+np.exp(-1*data))
    
    
def _correlation(data1,data2,window):
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data1))<=2 or len(np.unique(data2))<=2:
        return np.zeros(len(data1))
    try:
        df=pd.DataFrame({'data1':data1,'data2':data2})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').apply(lambda df : df.data1.rolling(window).corr(df.data2)) # 因为code排序是按顺序的所以可以用apply
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data1))
    
    
def _covariance(data1,data2,window):
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data1))<=2 or len(np.unique(data2))<=2:
        return np.zeros(len(data1))
    try:
        df=pd.DataFrame({'data1':data1,'data2':data2})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').apply(lambda df : df.data1.rolling(window).cov(df.data2)) # 因为code排序是按顺序的所以可以用apply
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data1))
    
    
def _scale(data):
    if len(np.unique(data))<=2:
        return np.zeros(len(data))
    else:
        return np.divide(data,np.sum(np.abs(data)))
    
    
def _decay_linear(data,window):  # 因子过去几天的加权平均值，权数随时间往前线性递减
    window=window[0]    
    if  window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        w=np.arange(1,window+1)
        w=w/np.sum(w)
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').data.transform(lambda x:x.rolling(window).apply(lambda s:np.dot(w,s.values)).values)
        return np.nan_to_num(value)
    except:
        return np.zeros(len(data))

In [22]:
Add=make_function(function=_Add,name='Add',arity=2)
Sub=make_function(function=_Sub,name='Sub',arity=2)
Mul=make_function(function=_Mul,name='Mul',arity=2)
Div=make_function(function=_Div,name='Div',arity=2)
Log=make_function(function=_Log,name='Log',arity=1)
Sqrt=make_function(function=_Sqrt,name='Sqrt',arity=1)
Inv=make_function(function=_Inv,name='Inv',arity=1)
ts_max=make_function(function=_ts_max,name='ts_max',arity=2) 
ts_min=make_function(function=_ts_min,name='ts_min',arity=2) 
ts_mid=make_function(function=_ts_mid,name='ts_mid',arity=2) 
ts_mean=make_function(function=_ts_mean,name='ts_mean',arity=2) 
ts_std=make_function(function=_ts_std,name='ts_std',arity=2)
delay=make_function(function=_delay,name='delay',arity=2)
delta=make_function(function=_delta,name='delta',arity=2)
rank=make_function(function=_rank,name='rank',arity=1)
sigmoid=make_function(function=_sigmoid,name='sigmoid',arity=1)
correlation=make_function(function=_correlation,name='correlation',arity=3)
covariance=make_function(function=_covariance,name='covariance',arity=3)
scale=make_function(function=_scale,name='scale',arity=1)
ts_rank=make_function(function=_ts_rank,name='ts_rank',arity=2)
ts_sum=make_function(function=_ts_sum,name='ts_sum',arity=2)
ts_product=make_function(function=_ts_product,name='ts_product',arity=2)

In [23]:
user_function=[ts_max,ts_min,ts_mid,ts_mean,ts_std,delay,delta,rank,correlation,ts_rank]
function_set=init_function+user_function

某时点某因子在全部股票暴露值排名与其下期回报排名的截面相关系数，它与IC的区别就是将因子的具体值以及收益的具体值，都转换为了所对应的数值在其截面上的排序名次。然后通过计算排序值的相关系数，得到RankIC。

In [24]:
import statsmodels.api as sm

def rankIC_metric(y,y_pred,w=None):
    
    def cal_rankIC(df):
        df.index=df.code
        # 去极值
        M=df.y_pred.mean()    #df.y_pred.median()
        M1=df.y_pred.std()    #(df.y_pred-M).abs().median()
        df.y_pred.loc[df.y_pred>M+3*M1]=M+3*M1    #上下剪枝限制极值范围
        df.y_pred.loc[df.y_pred<M-3*M1]=M-3*M1
        # 行业市值中性化
        day=df.date.iloc[0]
        ind=inds[day].copy()
        ind['mkv']=df.loc[ind.index,'mkv']
        input_x=ind.dropna().astype(float)
        input_y=df.loc[input_x.index,'y_pred']
        y_true=df.loc[input_x.index,'y'].values
        if len(input_x)==0:
            return 0
        else:
            mod=sm.OLS(input_y,input_x)
            res=mod.fit()
            y_adj=res.resid  #拟合残差
            #标准化
            y_adj=(y_adj-y_adj.mean())/y_adj.std()
        
            if len(np.unique(y_adj.values)) <=2:
                return 0
            else:
                # one = np.argsort(np.argsort(np.nan_to_num(y_adj.values))) 
                # two = np.argsort(np.argsort(np.nan_to_num(y_true)))
                # #return y_adj.corr(input_xy.mkt_delay,method='spearman')
                # return spearmanr(one,two)[0]  #为什么这个rankic与定义不同，并没有对于原有的数据进行排序
                return spearmanr(np.nan_to_num(y_adj.values),np.nan_to_num(y_true))[0]
            
    if len(np.unique(y_pred))<=2:
        return 0
    else:
        try:
            data=pd.DataFrame({'y_pred':y_pred,'y':y,'mkv':np.log(mkv_s),'code':test_code,'date':test_date})
            res=data.groupby('date').apply(cal_rankIC)
            return np.abs(np.nanmean(res))
        except:
            return 0

In [25]:
rankIC=make_fitness(function=rankIC_metric,greater_is_better=True)

In [26]:
test_gp=SymbolicTransformer(feature_names = fields,
                            function_set = function_set, #所有算子
                            generations = 3, #进化代数
                            population_size = 100, #种群规模
                            tournament_size = 10, #竞标赛规模
                            p_crossover=0.3,
                            p_subtree_mutation=0.05,
                            p_hoist_mutation=0.01,
                            p_point_mutation=0.03,
                            p_point_replace=0.35,
                            init_depth=(1,4),
                            const_range = None,
                            metric=rankIC,
                            parsimony_coefficient = 'auto',
                            low_memory=True,
                            verbose=2,
                            n_jobs = 6)

In [27]:
test_gp.fit(np.nan_to_num(stock_test.loc[:,fields].values),np.nan_to_num(test_mkt))

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:   21.6s remaining:   43.3s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:   40.4s finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


   0     8.11                0       13                0              N/A      1.35m


[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:    3.9s remaining:    7.8s


KeyboardInterrupt: 

In [28]:
print(stock_test.loc[:,fields].values.shape)

(1169858, 11)


In [31]:
print(np.nan_to_num(test_mkt).shape)

array([0.06566347, 0.05034483, 0.01690331, ...,        nan,        nan,
              nan])

In [30]:
stock_test.loc[:,fields]

,open,close,high,low,vol,amount,1,5,20,40,60
0,1593.86740,1596.05078,1603.69261,1573.12529,924852.96,1.468381e+08,1,5,20,40,60
1,1584.04219,1582.95050,1586.22557,1561.11670,1339495.24,2.109127e+08,1,5,20,40,60
2,1593.86740,1614.60951,1630.98486,1589.50064,1407507.44,2.267652e+08,1,5,20,40,60
3,1614.60951,1612.42613,1617.88458,1593.86740,1070503.21,1.717478e+08,1,5,20,40,60
4,1605.87599,1599.32585,1624.43472,1594.95909,1013205.28,1.628068e+08,1,5,20,40,60
...,...,...,...,...,...,...,...,...,...,...,...
1169853,NaN,NaN,NaN,NaN,NaN,NaN,1,5,20,40,60
1169854,NaN,NaN,NaN,NaN,NaN,NaN,1,5,20,40,60
1169855,NaN,NaN,NaN,NaN,NaN,NaN,1,5,20,40,60
1169856,NaN,NaN,NaN,NaN,NaN,NaN,1,5,20,40,60
